## 0 Import depencies

In [2]:
import mediapipe as mp
import cv2

In [3]:
mp_drawing = mp.solutions.drawing_utils
mp_holistic = mp.solutions.holistic

In [ ]:
mp_holistic.HAND_CONNECTIONS

## 1 Make detections

In [16]:
cap = cv2.VideoCapture(0)

with mp_holistic.Holistic(min_detection_confidence=0.5,min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():
        ret, frame = cap.read()

        # Recolor feed
        image = cv2.cvtColor(frame,cv2.COLOR_BGR2RGB)
        image.flags.writeable = False
        # Make detections
        results = holistic.process(image)

        image.flags.writeable=True
        image = cv2.cvtColor(image,cv2.COLOR_RGB2BGR)

        # 1. Draw face landmarks
        mp_drawing.draw_landmarks(image,results.face_landmarks,mp_holistic.FACEMESH_TESSELATION,
                              mp_drawing.DrawingSpec(color=(255,0,0),thickness=1,circle_radius=1),
                              mp_drawing.DrawingSpec(color=(240,0,0),thickness=1,circle_radius=1))

        # 2. Left hand landmarks
        mp_drawing.draw_landmarks(image,results.left_hand_landmarks,mp_holistic.HAND_CONNECTIONS,
                              mp_drawing.DrawingSpec(color=(255,0,0),thickness=2,circle_radius=2),
                              mp_drawing.DrawingSpec(color=(240,0,0),thickness=2,circle_radius=2))

        # 3. Right hand landmarks
        mp_drawing.draw_landmarks(image,results.right_hand_landmarks,mp_holistic.HAND_CONNECTIONS,
                              mp_drawing.DrawingSpec(color=(255,0,0),thickness=2,circle_radius=2),
                              mp_drawing.DrawingSpec(color=(240,0,0),thickness=2,circle_radius=2))

        # 4. Pose detection
        mp_drawing.draw_landmarks(image,results.pose_landmarks,mp_holistic.HAND_CONNECTIONS,
                              mp_drawing.DrawingSpec(color=(255,0,0),thickness=2,circle_radius=2),
                              mp_drawing.DrawingSpec(color=(240,0,0),thickness=2,circle_radius=2))

        cv2.imshow("Holistic model detection",image)

        if cv2.waitKey(10) & 0xFF==ord('q'):
            break
cap.release()
cv2.destroyAllWindows()            

## 2 Capture landmarks and Export to CSV

In [29]:
import os,sys
import numpy as np

In [33]:
num_coords = len(results.pose_landmarks.landmark) + len(results.face_landmarks.landmark)
num_coords

501

In [69]:
landmarks = ['class']
for val in range(1,num_coords+1):
    landmarks+=[f"x{val}, y{val}, z{val}, v{val}"] 

In [90]:
import csv
with open('coords.csv',mode="w",newline="") as f:
    csv_writer = csv.writer(f,delimiter=",",quotechar="'",quoting=csv.QUOTE_MINIMAL)
    csv_writer.writerow(landmarks)

In [95]:
class_name = "Victorius"

In [96]:
cap = cv2.VideoCapture(0)

with mp_holistic.Holistic(min_detection_confidence=0.5,min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():
        ret, frame = cap.read()

        # Recolor feed
        image = cv2.cvtColor(frame,cv2.COLOR_BGR2RGB)
        image.flags.writeable = False
        # Make detections
        results = holistic.process(image)

        image.flags.writeable=True
        image = cv2.cvtColor(image,cv2.COLOR_RGB2BGR)

        # 1. Draw face landmarks
        mp_drawing.draw_landmarks(image,results.face_landmarks,mp_holistic.FACEMESH_TESSELATION,
                              mp_drawing.DrawingSpec(color=(255,0,0),thickness=1,circle_radius=1),
                              mp_drawing.DrawingSpec(color=(240,0,0),thickness=1,circle_radius=1))

        # 2. Left hand landmarks
        mp_drawing.draw_landmarks(image,results.left_hand_landmarks,mp_holistic.HAND_CONNECTIONS,
                              mp_drawing.DrawingSpec(color=(255,0,0),thickness=2,circle_radius=2),
                              mp_drawing.DrawingSpec(color=(240,0,0),thickness=2,circle_radius=2))

        # 3. Right hand landmarks
        mp_drawing.draw_landmarks(image,results.right_hand_landmarks,mp_holistic.HAND_CONNECTIONS,
                              mp_drawing.DrawingSpec(color=(255,0,0),thickness=2,circle_radius=2),
                              mp_drawing.DrawingSpec(color=(240,0,0),thickness=2,circle_radius=2))

        # 4. Pose detection
        mp_drawing.draw_landmarks(image,results.pose_landmarks,mp_holistic.HAND_CONNECTIONS,
                              mp_drawing.DrawingSpec(color=(255,0,0),thickness=2,circle_radius=2),
                              mp_drawing.DrawingSpec(color=(240,0,0),thickness=2,circle_radius=2))
        
        # Export coordinates
        try:
            # Extract pose landmarks
            pose = results.pose_landmarks.landmark
            pose_row = list(np.array([[landmark.x,landmark.y,landmark.z,landmark.visibility] for landmark in pose]).flatten())

            # Extract face landmarks
            face = results.face_landmarks.landmark
            face_row = list(np.array([[landmark.x,landmark.y,landmark.z,landmark.visibility] for landmark in face]).flatten())

            # Concat rows
            row = pose_row+face_row

            # Insert row
            row.insert(0,class_name)

            # Export to csv
            with open('coords.csv',mode='a',newline='') as f:
                csv_writer = csv.writer(f,delimiter=",",quotechar="'",quoting=csv.QUOTE_MINIMAL)
                csv_writer.writerow(row)

        except Exception as e:
            print(e)

        cv2.imshow("Holistic model detection",image)

        if cv2.waitKey(10) & 0xFF==ord('q'):
            break
cap.release()
cv2.destroyAllWindows()            

## 3 Train custom model using scikit-learn

In [134]:
from sklearn.model_selection import train_test_split
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression,RidgeClassifier
from sklearn.pipeline import make_pipeline
from sklearn.ensemble import RandomForestClassifier,GradientBoostingClassifier

### 3.1 Read in collected data and process

In [105]:
df = pd.read_csv('coords.csv')
df.sample(5)

,class,'x1,y1,z1,v1','x2,y2,z2,v2','x3,...,z499,v499','x500,y500,z500,v500','x501,y501,z501,v501'
382,Sad,0.597149,0.267684,-1.323826,0.999840,0.631975,0.196773,-1.302478,0.999670,0.653470,...,-0.011770,0.0,0.654806,0.184508,0.007915,0.0,0.659594,0.177332,0.008632,0.0
109,Happy,0.554802,0.296743,-0.583323,0.999993,0.570428,0.250504,-0.539060,0.999988,0.583454,...,-0.003003,0.0,0.600070,0.241152,0.012137,0.0,0.603935,0.235614,0.013085,0.0
525,Victorius,0.552861,0.335191,-0.444129,0.999996,0.566966,0.307970,-0.357046,0.999990,0.576137,...,0.008749,0.0,0.593748,0.322887,0.026123,0.0,0.596545,0.319461,0.027483,0.0
111,Happy,0.555153,0.298754,-0.576781,0.999993,0.571414,0.252038,-0.533597,0.999988,0.584845,...,-0.004757,0.0,0.601112,0.243528,0.010080,0.0,0.604873,0.238695,0.010849,0.0
338,Sad,0.575909,0.199637,-0.949337,0.999903,0.590780,0.154384,-0.886057,0.999840,0.601955,...,0.013312,0.0,0.618516,0.162212,0.031739,0.0,0.622815,0.155894,0.033156,0.0


In [115]:
X=df.iloc[:,1:]
y=df.iloc[:,0]

In [132]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.3,random_state=42)

### 3.2 Train machine learning classification model

In [135]:
pipelines = {
    "lr":make_pipeline(StandardScaler(),LogisticRegression()),
    "rc":make_pipeline(StandardScaler(),RidgeClassifier()),
    "rf":make_pipeline(StandardScaler(),RandomForestClassifier()),
    "gb":make_pipeline(StandardScaler(),GradientBoostingClassifier())
}

In [147]:
list(pipelines.values())[0]

Pipeline(steps=[('standardscaler', StandardScaler()),
                ('logisticregression', LogisticRegression())])